# Prague Pedestrian Accessibility for Children (age 10-16)

### Installing packages

I will use OSMnx - a Python package by Geoff Boeing to work with street networks: retrieve, construct, analyze, and visualize street networks (and more) from OpenStreetMap.
<a>https://github.com/gboeing/osmnx</a>.

In [2]:
!conda install -c conda-forge/label/gcc7 osmnx

Solving environment: done

# All requested packages already installed.



Installing gecoder. A simple and consistent geocoding library

In [3]:
!conda install -c conda-forge geocoder

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    psycopg2-2.8.2             |   py36h72c5cf5_0         163 KB  conda-forge
    gdal-2.2.2                 |   py36hc209d97_1         767 KB
    libssh2-1.8.2              |       h22169c7_2         257 KB  conda-forge
    libcurl-7.65.3             |       h20c2e04_0         588 KB
    libpq-11.2                 |       h20c2e04_0         2.7 MB
    fiona-1.7.13               |   py36hb00a9d7_1         1.8 MB  conda-forge
    freetds-1.1rc3             |       h4fe99da_0         2.4 MB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    hdf5-1.10.1                |                2         5.0 MB  conda-forge
    pycurl-7.43.0.3            |   py36h16ce93b_0    

In [4]:
!conda install -c conda-forge pandana

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - pandana


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    osmnet-0.1.5               |             py_3          29 KB  conda-forge
    pandana-0.4.4              |   py36hb3f55d8_0         158 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         187 KB

The following NEW packages will be INSTALLED:

    osmnet:  0.1.5-py_3           conda-forge
    pandana: 0.4.4-py36hb3f55d8_0 conda-forge


osmnet-0.1.5         | 29 KB     | ##################################### | 100% 
pandana-0.4.4        | 158 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [5]:
!conda update --all

Solving environment: \ 
  - defaults::libgfortran-3.0.0-1
  - conda-forge::libgfortran-3.0.0done

## Package Plan ##

  environment location: /opt/conda/envs/Python36


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    xlsxwriter-1.2.1           |             py_0         106 KB
    soupsieve-1.9.3            |           py36_0          60 KB
    beautifulsoup4-4.8.0       |           py36_0         147 KB
    mkl_fft-1.0.14             |   py36ha843d7b_0         173 KB
    fiona-1.8.4                |   py36hc38cc03_0         1.0 MB
    openjpeg-2.3.0             |       h05c96fa_1         456 KB
    jeepney-0.4.1              |             py_0          21 KB
    jupyterlab_server-1.0.6    |             py_0          26 KB
    fastcache-1.1.0            |   py36h7b6447c_0          31 KB
    dask-2.6.0                 |             py_0          12 KB
    markupsafe-1.1.1           |   py

Importing all nessecary libraries

In [6]:
import osmnx as ox
import io
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import folium

import json
import urllib.request
import requests
from pandas.io.json import json_normalize
import operator
from shapely.geometry import Polygon
import geopandas as gpd
from pandana.loaders import osm


## Preparing envieroment 

In [65]:
# The code was removed by Watson Studio for sharing.

In [94]:
from ibm_botocore.client import Config
import ibm_boto3

def upload_file(credentials,local_file_name,key): 
    storage = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['apikey'],
    ibm_service_instance_id=credentials['iam_serviceid_crn'],
    ibm_auth_endpoint=credentials['auth_ep'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ep'])
    
    try:
        res=storage.upload_file(Filename=local_file_name, Bucket=credentials['bucket'],Key=key)
    except Exception as e:
        print(Exception, e)
    else:
        print('File {} Uploaded'.format(local_file_name))
        
def download_file(credentials,local_file_name,key):  
    storage = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['apikey'],
    ibm_service_instance_id=credentials['iam_serviceid_crn'],
    ibm_auth_endpoint=credentials['auth_ep'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ep'])
    try:
        res= storage.download_file(Bucket=credentials['bucket'],Key=key,Filename=local_file_name)
    except Exception as e:
        print(Exception, e)
    else:
        print('File {} Downloaded'.format(local_file_name))

In [89]:
import geocoder

def get_coordinates(dataFrame, index_row):
    dict_coordinates = {}
    for index, row in dataFrame.iterrows():
        try:
            g = geocoder.arcgis(row[index_row])
            lat = g.json['lat']
            lng = g.json['lng']
            dict_coordinates[index] = [lat, lng]
        except:
            print ('Failed to get coordinates for {}: {}'.format(index_row, sys.exc_info()[0]))
    
    dataFrame['latitude'] = 0.0
    dataFrame['longitude'] = 0.0
    
    for k, v in dict_coordinates.items():
        dataFrame.loc[dataFrame.index == k,'latitude']=v[0]
        dataFrame.loc[dataFrame.index == k,'longitude']=v[1]
    print('Done')

## Data acquisition and cleaning

As main data source I sellected <a>http://opendata.praha.eu</a>.  This is a big database of data of different types amd it contains data from different fileds: transport, society, ecology, population e.t.c  
This data sets were mainly colleted and structured by Prague Institute of Planning and Development <a>www.iprpraha.cz</a> For my project I mainly intrested in:
* Shape and location of Prague administrative districs
* Districts population
* Geo location different types of social infrastructure 

### Districts borders and population.

First step is to determine the shape and location of the administrative disctrics of Prague.

In [67]:
mestky_casty_url = 'http://opendata.iprpraha.cz/CUR/DTMP/TMMESTSKECASTI_P/WGS_84/TMMESTSKECASTI_P.json'
results = requests.get(mestky_casty_url).json(encoding = "utf8")
mestky_casty = json_normalize(results['features']) 
mestky_casty.head()

,type,geometry.type,geometry.coordinates,properties.OBJECTID,properties.DAT_VZNIK,properties.DAT_ZMENA,properties.PLOCHA,properties.ID,properties.KOD_MC,properties.NAZEV_MC,properties.KOD_MO,properties.KOD_SO,properties.TID_TMMESTSKECASTI_P,properties.POSKYT,properties.ID_POSKYT,properties.STAV_ZMENA,properties.NAZEV_1,properties.Shape_Length,properties.Shape_Area
0,Feature,Polygon,"[[[14.533725418000074, 50.16223134300003], [14...",1,20181106141412,20190423111436,10183715.88,25,547310,Praha-Čakovice,94,221,25,HMP-IPR,43,U,Čakovice,0.213162,1.018372e+07
1,Feature,Polygon,"[[[14.293206908000059, 50.07751405400006], [14...",2,20181106141412,20181106164427,3253142.41,52,547174,Praha 17,60,213,52,HMP-IPR,43,U,Praha 17,0.095029,3.253142e+06
2,Feature,Polygon,"[[[14.483934895000061, 49.99241857800007], [14...",3,20181009145125,20190821104230,5234736.54,19,547051,Praha-Libuš,43,124,19,HMP-IPR,43,U,Libuš,0.200404,5.234737e+06
3,Feature,Polygon,"[[[14.506905018000054, 50.17143575600005], [14...",4,20170817145228,20170818091113,3380681.90,35,538124,Praha-Březiněves,86,86,35,HMP-IPR,43,U,Březiněves,0.127235,3.380682e+06
4,Feature,Polygon,"[[[14.43852135000003, 50.06691477800007], [14....",5,20180910110223,20180910113234,4184937.95,30,500089,Praha 2,27,27,30,HMP-IPR,43,U,Praha 2,0.134652,4.184938e+06


In [68]:
mestky_casty.shape

(57, 19)

In [69]:
geo_unique = np.array(mestky_casty['properties.NAZEV_MC'].apply(lambda x: x.lower()).unique())
geo_unique

array(['praha-čakovice', 'praha 17', 'praha-libuš', 'praha-březiněves',
       'praha 2', 'praha 1', 'praha 11', 'praha-zbraslav', 'praha 15',
       'praha 4', 'praha 5', 'praha 20', 'praha-dolní měcholupy',
       'praha 6', 'praha 9', 'praha 10', 'praha 14', 'praha 12',
       'praha-kolovraty', 'praha-újezd', 'praha 13', 'praha-řeporyje',
       'praha-suchdol', 'praha-ďáblice', 'praha-šeberov',
       'praha-dolní chabry', 'praha 19', 'praha-koloděje',
       'praha-satalice', 'praha-petrovice', 'praha 3',
       'praha-velká chuchle', 'praha-dolní počernice',
       'praha-přední kopanina', 'praha-královice', 'praha-kunratice',
       'praha-slivenec', 'praha-vinoř', 'praha-lochkov', 'praha-nebušice',
       'praha-benice', 'praha 18', 'praha-křeslice', 'praha-troja',
       'praha 7', 'praha-nedvězí', 'praha 21', 'praha-běchovice',
       'praha-štěrboholy', 'praha-dubeč', 'praha-lysolaje',
       'praha-lipence', 'praha 8', 'praha 22', 'praha-zličín', 'praha 16',
       'praha-

Districts population

In [70]:
url_population =  'https://www.czso.cz/documents/10180/25233177/sldb_zv.csv'
df_population = pd.read_csv(url_population,encoding = "ISO 8859-2")
df_population.head()

,typuz_naz,nazev,uzcis,uzkod,u01,u02,u03,u04,u05,u06,u07,u08,u09,u10,u11
0,kraj,Hlavní město Praha,100,3018,1268796.0,613738.0,655058.0,153622.0,908321.0,201029.0,644643.0,600730.0,92927.0,542168.0,579509.0
1,kraj,Středočeský kraj,100,3026,1289211.0,637252.0,651959.0,199300.0,895024.0,190911.0,639851.0,587539.0,286780.0,482860.0,523045.0
2,kraj,Jihočeský kraj,100,3034,628336.0,308296.0,320040.0,91119.0,435187.0,100000.0,307130.0,280844.0,123048.0,247608.0,262692.0
3,kraj,Plzeňský kraj,100,3042,570401.0,282137.0,288264.0,79469.0,396468.0,92734.0,278674.0,255278.0,105835.0,226298.0,242397.0
4,kraj,Karlovarský kraj,100,3051,295595.0,145483.0,150112.0,42159.0,207480.0,44538.0,139871.0,123100.0,39845.0,119403.0,128904.0


Cleaning population data set. For my project only make sence data 

In [71]:
df_population = df_population[(df_population.uzcis == 44)& (df_population.nazev.str.find('Praha') != -1)][['nazev','u01','u04', 'u05', 'u06']]
df_population.rename(columns={'nazev':'Name','u01':'Total', 'u04':'Kids', 'u05':'Middle', 'u06':'Senior'}, inplace = True)
df_population['Name'] = df_population['Name'].map(lambda x: x.lower())
df_population.shape

(57, 5)

Quick analizing of distrcit population

In [72]:
population_unique = df_population['Name'].unique()
population_unique

array(['praha 1', 'praha 2', 'praha 3', 'praha 4', 'praha 5', 'praha 6',
       'praha 7', 'praha 8', 'praha 9', 'praha 10', 'praha-běchovice',
       'praha-benice', 'praha-březiněves', 'praha-dolní počernice',
       'praha-dubeč', 'praha 20', 'praha-klánovice', 'praha-koloděje',
       'praha-kolovraty', 'praha-královice', 'praha-křeslice',
       'praha-nedvězí', 'praha-satalice', 'praha 22', 'praha 21',
       'praha-vinoř', 'praha-lipence', 'praha-lochkov',
       'praha-přední kopanina', 'praha 16', 'praha-řeporyje',
       'praha-slivenec', 'praha 13', 'praha-\x8aeberov', 'praha-újezd',
       'praha-zbraslav', 'praha-zličín', 'praha 11', 'praha-kunratice',
       'praha-libu\x9a', 'praha 12', 'praha-velká chuchle',
       'praha-lysolaje', 'praha-nebu\x9aice', 'praha 17', 'praha-suchdol',
       'praha-ďáblice', 'praha-dolní chabry', 'praha-čakovice',
       'praha-troja', 'praha 19', 'praha 14', 'praha-dolní měcholupy',
       'praha 15', 'praha-petrovice', 'praha-\x8atěrboho

Checking the difference between to datasets

In [73]:
districts_diff_geo = list(set(geo_unique)-set(population_unique))
districts_diff_geo

['praha-šeberov', 'praha-štěrboholy', 'praha-libuš', 'praha-nebušice']

In [74]:
districts_diff_pop = list(set(population_unique)-set(geo_unique))
districts_diff_pop

['praha-libu\x9a',
 'praha-nebu\x9aice',
 'praha-\x8aeberov',
 'praha-\x8atěrboholy']

Population data set has encoding errors. Lets fix it

In [75]:
df_population.loc[df_population.Name == 'praha-libu\x9a', 'Name'] = 'praha-libuš'
df_population.loc[df_population.Name == 'praha-\x8aeberov', 'Name'] = 'praha-šeberov'
df_population.loc[df_population.Name == 'praha-nebu\x9aice', 'Name'] = 'praha-nebušice'
df_population.loc[df_population.Name == 'praha-\x8atěrboholy', 'Name'] = 'praha-štěrboholy'

In [76]:
population_unique = df_population['Name'].unique()
districts_diff_pop = list(set(population_unique)-set(geo_unique))
districts_diff_pop

[]

Districts dataset and population dataset have the same districs. Lets make a join of two data sets

In [77]:
result = []

result.append([
    v['properties']['NAZEV_MC'].lower(),
    v['geometry']['coordinates'][0],
    v['properties']['PLOCHA']] for v in results['features'])
    
df_prague_districts = pd.DataFrame([item for result in result for item in result])
df_prague_districts.columns = ['Name', 'Geometry', 'Area']

In [79]:
df_prague = df_prague_districts.set_index('Name').join(df_population.set_index('Name'))
quotient = df_prague['Middle']/1000
df_prague['Kids_per_1000'] = df_prague['Kids']/quotient
df_prague.sort_values('Name', inplace = True)
df_prague.reset_index(inplace=True)

In [80]:
df_prague.shape

(57, 8)

In [90]:
get_coordinates(df_prague, 'Name')

Done


In [91]:
df_prague.head()

,Name,Geometry,Area,Total,Kids,Middle,Senior,Kids_per_1000,latitude,longitude
0,praha 1,"[[14.410891049000043, 50.078674687000046], [14...",5538443.86,30561.0,2391.0,22963.0,4594.0,104.124026,50.08728,14.41742
1,praha 10,"[[14.531321086000048, 50.072240288000046], [14...",18599366.98,113200.0,12213.0,76625.0,23937.0,159.386623,50.06762,14.46016
2,praha 11,"[[14.54355294800007, 50.03618763800006], [14.5...",9793679.84,75741.0,8688.0,54983.0,11816.0,158.012477,50.03178,14.50719
3,praha 12,"[[14.450632163000023, 50.01452735600003], [14....",23317909.06,53515.0,6156.0,39699.0,7480.0,155.066878,50.00564,14.40462
4,praha 13,"[[14.320621949000042, 50.04010680700003], [14....",13196802.19,59906.0,7985.0,46514.0,5109.0,171.668745,50.05163,14.34231


Saving data set to stroage for later use

In [95]:
file_name = 'prague_district_population.csv'
df_prague.to_csv(file_name)
upload_file(storage_creds,file_name,file_name)

File prague_district_population.csv Uploaded


Explore children popupualtion in Prague

In [23]:
polygon = [Polygon(x) for x in df_prague.Geometry]
crs = {'init': 'epsg:4326'}
gdf = gpd.GeoDataFrame(df_prague, crs=crs, geometry=polygon)

In [24]:
gdf.head()

,Geometry,Area,Total,Kids,Middle,Senior,Kids_per_1000,latitude,longitude,geometry
Name,,,,,,,,,,
praha 1,"[[14.410891049000043, 50.078674687000046], [14...",5538443.86,30561.0,2391.0,22963.0,4594.0,104.124026,50.4669,4.86746,"POLYGON ((14.41089104900004 50.07867468700005,..."
praha 10,"[[14.531321086000048, 50.072240288000046], [14...",18599366.98,113200.0,12213.0,76625.0,23937.0,159.386623,50.4669,4.86746,"POLYGON ((14.53132108600005 50.07224028800005,..."
praha 11,"[[14.54355294800007, 50.03618763800006], [14.5...",9793679.84,75741.0,8688.0,54983.0,11816.0,158.012477,50.4669,4.86746,"POLYGON ((14.54355294800007 50.03618763800006,..."
praha 12,"[[14.450632163000023, 50.01452735600003], [14....",23317909.06,53515.0,6156.0,39699.0,7480.0,155.066878,50.4669,4.86746,"POLYGON ((14.45063216300002 50.01452735600003,..."
praha 13,"[[14.320621949000042, 50.04010680700003], [14....",13196802.19,59906.0,7985.0,46514.0,5109.0,171.668745,50.4669,4.86746,"POLYGON ((14.32062194900004 50.04010680700003,..."


In [25]:
gdf_kids = gdf[['Kids_per_1000', 'geometry']]
gdf_kids.reset_index(inplace = True)


### Points of interest

#### Playgrounds
Data from Hřiště Praha 2014 - 2016 <a>http://www.hristepraha.cz</a> Last update 19.01.2018

In [96]:
url_playgrounds = 'http://opendata.praha.eu/dataset/3c3ca9ca-fbc0-4f97-b624-ed967f5d9a24/resource/e19c2e29-5e33-4449-8847-5dc8f5b8a2f2/download/db144c03-1a0f-456f-a32b-9c48ccfc0813-playgrounds.json'
results = requests.get(url_playgrounds).json(encoding = "utf8")
df_playgrounds = json_normalize(results['features']) 
df_playgrounds.head()

,type,properties.id,properties.name,properties.url,properties.perex,properties.content,properties.district,properties.address,properties.properties,properties.image.url,geometry.type,geometry.coordinates
0,Feature,101,Sídliště Petrovice - Rezlerova,http://www.hristepraha.cz/hriste/mapa/sidliste...,"Lokalita nabízí několik pěkných menších hřišť,...",Za panelovým domem v Rezlerově ulici se rozklá...,praha-petrovice,"Rezlerova 278, 109 00 Praha-Praha-Petrovice, Č...",[],http://www.hristepraha.cz/images/img/41f5da50e...,Point,"[14.56323719, 50.038024902]"
1,Feature,43,Bohnice a Čimice - Čimice,http://www.hristepraha.cz/hriste/mapa/bohnice-...,Nedaleko od sebe leží 2 pěkná hřiště.,"Větší hřiště se rozkládá, mezi ulicemi Toruňsk...",praha-8,"Skálova 545/24, Čimice, 181 00 Praha-Praha 8, ...",[],http://www.hristepraha.cz/images/img/b07bef69a...,Point,"[14.438850403, 50.13401413]"
2,Feature,100,Na Krejcárku - hřiště 60.B,http://www.hristepraha.cz/hriste/mapa/na-krejc...,Lokalita se skvěle hodí pro rodiny s dětmi růz...,Dětské hřiště (60.A) najdete na konci ulice St...,praha-3,"Za Žižkovskou vozovnou 2716/19, Žižkov, 130 00...",[],http://www.hristepraha.cz/images/img/a0cad32d8...,Point,"[14.476410866, 50.094387054]"
3,Feature,131,Uhříněves - hřiště 82.B,http://www.hristepraha.cz/hriste/mapa/uhrineve...,"Hřiště, lesopark a další zajímavá místa, to je...","Cestu doporučujeme zahájit na Novém náměstí, k...",praha-22,"V Bytovkách 754/30, Uhříněves, 104 00 Praha-Pr...",[],http://www.hristepraha.cz/images/img/3063cb73f...,Point,"[14.593131065, 50.036453247]"
4,Feature,72,Hostivařský lesopark (východní část) - hřiště ...,http://www.hristepraha.cz/hriste/mapa/hostivar...,Trasa je vhodným polodenním rodinným výletem.,Popis: Asi 300 m od prodejny Lidl v Hornoměcho...,praha-15,"U Břehu 1111, Hostivař, 102 00 Praha-Praha 15,...",[],http://www.hristepraha.cz/images/img/2d73f6832...,Point,"[14.539891243, 50.043731689]"


In [97]:
poi_type = 'playground'
result = []

result.append([
    poi_type,
    v['properties']['district'].lower(),
    v['geometry']['coordinates'][0],
    v['geometry']['coordinates'][1]] for v in results['features'])
    
df_prague_poi = pd.DataFrame([item for result in result for item in result])
df_prague_poi.columns = ['Type', 'District_Name', 'Lat','Lng']
df_prague_poi.head()

,Type,District_Name,Lat,Lng
0,playground,praha-petrovice,14.563237,50.038025
1,playground,praha-8,14.438850,50.134014
2,playground,praha-3,14.476411,50.094387
3,playground,praha-22,14.593131,50.036453
4,playground,praha-15,14.539891,50.043732


In [98]:
df_prague_poi.shape

(145, 4)

### Sport facilities
Magistrát hl. m. Prahy 	1. duben 2019, 0:00 (UTC+02:00)

In [99]:
url_sport = 'http://opendata.praha.eu/datastore/dump/5d1ee13f-f6e9-4ee9-a1bd-48d5ca2bb867?format=json'
results = requests.get(url_sport).json(encoding = "utf8")
result = []

result.append([
    v[2],
    'praha {}'.format(v[6]),
    'sport'] for v in results['records'])
    
df_sport = pd.DataFrame([item for result in result for item in result])
df_sport.columns = ['Address', 'Type', 'District_Name']
df_sport.head()

,Address,Type,District_Name
0,"Butovická 837/41, Praha 5",praha 5,sport
1,"Zahradníčkova, Praha 5, 150 00",praha 5,sport
2,"Senovážné náměstí 6, Praha 1, 110 00",praha 1,sport
3,"Zelenkova 3/530, Praha 12, 142 00",praha 12,sport
4,"Štvanice 38, Praha 7, 170 00",praha 7,sport


In [100]:
df_sport.isnull().values.any()

False

In [ ]:
get_coordinates(df_sport, 'Address')
df_sport.head()    

In [32]:
df_sport['Type'] = 'sport'
df_sport['prague'] = df_sport['prague'].map(lambda x: 'praha {}'.format(x))
df_sport.rename(columns={'prague':'District_Name'}, inplace=True)
df_sport.drop(columns=['title', 'address', 'phone','web', 'Chmelnice, Praha'], inplace = True)
df_sport.head()

KeyError: 'prague'

### Libraries

In [ ]:
url_libs = 'https://cs.wikipedia.org/wiki/M%C4%9Bstsk%C3%A1_knihovna_v_Praze'

f = urllib.request.urlopen(url_libs)
html = f.read()

try: 
    from BeautifulSoup import BeautifulSoup
except ImportError:
    from bs4 import BeautifulSoup

parsed_html = BeautifulSoup(html)
tag_header = parsed_html.find_all('h4')
district_tags = []

for tag in tag_header:
    tag_match = False
    district =''
    for child in tag.children:
        if child.get("class")[0] == 'mw-headline':
            district = child.get_text().lower()
            tag_match = True
    if tag_match == True:
        nextsibling = tag.next_sibling
        while  True:
            if nextsibling.find('ul') != -1 :
                lists = nextsibling.find_all('li')
                for lib in  lists:
                    district_tags.append([lib.get_text(), 'library', district])
                break
            else:
                nextsibling = nextsibling.next_sibling

df_libs = pd.DataFrame(data=district_tags)
df_libs.columns = ['Address', 'Type', 'District_Name']

In [ ]:
get_coordinates(df_libs, 'Address')
df_libs.head()